### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2022
quarter = 2

today = date.today()
today

datetime.date(2022, 7, 15)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s)
OR (year = %s-1 AND quarter >= %s+1) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp

,name,year,quarter,q_amt,Counter
10,AOT,8086,10,-16784073,4
79,GVREIT,8086,10,671535,4
129,OISHI,8086,10,834166,4
189,TFFIF,8086,10,1209444,4
196,TISCO,8086,10,6995711,4


In [3]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 2-1) 
OR (year = 2022-1 AND quarter >= 2) 
ORDER BY year DESC, quarter DESC


In [4]:
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(212, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,AOT,2022,Q2,-16784073,-17151697,367624,2.14
1,GVREIT,2022,Q2,671535,672205,-670,-0.10
2,OISHI,2022,Q2,834166,709629,124537,17.55
3,TFFIF,2022,Q2,1209444,1138781,70663,6.21
4,TISCO,2022,Q2,6995711,6813664,182047,2.67


In [6]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year,quarter)
print(sql)


DELETE FROM qt_profits 
WHERE year = 2022 AND quarter = 'Q2'


In [7]:
rp = conlt.execute(sql)
rp.rowcount

0

In [8]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
tickers.shape

(401, 2)

In [9]:
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
df_ins.dtypes

name           object
year            int64
quarter        object
q_amt_c         int64
q_amt_p         int64
inc_profit      int64
Pct           float64
id              int64
dtype: object

In [10]:
rcds = df_ins.values.tolist()
len(rcds)

5

In [11]:
for rcd in rcds:
    print(rcd)

['AOT', 2022, 'Q2', -16784073, -17151697, 367624, 2.14, 24]
['GVREIT', 2022, 'Q2', 671535, 672205, -670, -0.1, 654]
['OISHI', 2022, 'Q2', 834166, 709629, 124537, 17.55, 338]
['TFFIF', 2022, 'Q2', 1209444, 1138781, 70663, 6.21, 686]
['TISCO', 2022, 'Q2', 6995711, 6813664, 182047, 2.67, 531]


In [12]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [13]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [14]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [15]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
2,OISHI,2022,Q2,834166,709629,124537,17.55
3,TFFIF,2022,Q2,1209444,1138781,70663,6.21
4,TISCO,2022,Q2,6995711,6813664,182047,2.67
1,GVREIT,2022,Q2,671535,672205,-670,-0.10


In [16]:
criteria_2 = df_ins.q_amt_p > 400000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
2,OISHI,2022,Q2,834166,709629,124537,17.55
3,TFFIF,2022,Q2,1209444,1138781,70663,6.21
4,TISCO,2022,Q2,6995711,6813664,182047,2.67
1,GVREIT,2022,Q2,671535,672205,-670,-0.10


In [17]:
criteria_3 = df_ins.Pct > 10
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
2,OISHI,2022,Q2,834166,709629,124537,17.55


In [18]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
2,OISHI,2022,Q2,834166,709629,124537,17.55
